<a href="https://colab.research.google.com/github/KingAbdulRehman/LangChain/blob/main/LangChain_ToolCalling/LangChain_Calculator_Tool_Calling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Instruction:

Setup Gemeni Api key as **"GOOGLE_API_KEY"** in Google Colab Secret


# Install and Import Packages

In [ ]:
!pip install langchain-google-genai langchain

In [16]:
from langchain_google_genai import ChatGoogleGenerativeAI
import os
from google.colab import userdata
from langchain_core.tools import tool
from langchain import hub
from langchain.schema import AIMessage, HumanMessage, SystemMessage, FunctionMessage
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.agents import initialize_agent, create_tool_calling_agent, AgentExecutor, AgentExecutorIterator
from langchain.agents.schema import AgentScratchPadChatPromptTemplate
from rich.console import Console
from rich.markdown import Markdown
from langchain_core.chat_history import InMemoryChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

# Setup API Keys

In [5]:
if "GOOGLE_API_KEY" not in os.environ:
  os.environ["GOOGLE_API_KEY"] = userdata.get("GOOGLE_API_KEY")

# Setup LLM

In [6]:
llm = ChatGoogleGenerativeAI(
  model="gemini-1.5-pro",
  max_tokens=1000,
  max_retries=3,
  top_k=2,
  temperature=0.8
)

In [7]:
# test llm
print(llm.invoke("Hello"))

content='Hello! How can I help you today?' additional_kwargs={} response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []} id='run-8d427391-b8eb-4438-8d12-3e9a56446890-0' usage_metadata={'input_tokens': 2, 'output_tokens': 10, 'total_tokens': 12, 'input_token_details': {'cache_read': 0}}


# Create Calculator Tool

In [65]:
class Calculator:
    def calculate(self, expression: str):
        try:
            result = eval(expression, {"__builtins__": None}, {})
            return result
        except Exception as e:
            return f"Error: {e}"


@tool
def calculator(expression: str):
    """A mathematical expression as a string (e.g., "2 + 2")."""
    calc = Calculator()
    result = calc.calculate(expression)
    return FunctionMessage(name="calculator", content=str(result))

In [62]:
# test tool
calculator("(2 + 2)-(6 + 4)")

FunctionMessage(content='-6', additional_kwargs={}, response_metadata={}, name='calculator')

# Create Tool Calling Agent

In [63]:
tools = [calculator]

prompt = hub.pull("hwchase17/openai-tools-agent")

agent = create_tool_calling_agent(
  llm=llm,
  tools=tools,
  prompt=prompt
)

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

/usr/local/lib/python3.11/dist-packages/langsmith/client.py:256: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [64]:
ai_search_res = agent_executor.invoke({"input": "Please solve: 2 * 5 + 7 / 18 and explain"})



> Entering new AgentExecutor chain...

Invoking: `calculator` with `{'expression': '2 * 5 + 7 / 18'}`


content='10.38888888888889' additional_kwargs={} response_metadata={} name='calculator'

ChatGoogleGenerativeAIError: Invalid argument provided to Gemini: 400 * GenerateContentRequest.contents[2].parts[0].function_response.name: Name cannot be empty.


In [ ]:
console = Console()
console.print(Markdown(ai_search_res["output"]))

# Add Chat history

In [19]:
memory = InMemoryChatMessageHistory(session_id="test-session")

agent_with_chat_history = RunnableWithMessageHistory(
    agent_executor,
    lambda session_id: memory,
    input_messages_key="input",
    history_messages_key="chat_history",
    output_messages_key="output"  # Ensure this key matches the output of your agent
)

config = {"configurable": {"session_id": "test-session"}}

In [20]:
print(
    agent_with_chat_history.invoke(
        {"input": "Please solve 10 * 20 * 100 / 150"}, config
    )["output"]
)



> Entering new AgentExecutor chain...

Invoking: `calculator` with `{'expression': '10 * 20 * 100 / 150'}`


133.33333333333334

ChatGoogleGenerativeAIError: Invalid argument provided to Gemini: 400 * GenerateContentRequest.contents[2].parts[0].function_response.name: Name cannot be empty.


In [13]:
print(
    agent_with_chat_history.invoke(
        {"input": "did you tell me what i ask previosuly?"}, config
    )["output"]
)

I am unable to track previous interactions, so I cannot say if I answered your previous question.
